# Visualize Weather Station Interactions on Map
This notebook contains code to load a trained encoder, predict latent interactions (probably on test set) and finally visualize this graph on a map.

In [1]:
# Install plotting library if not already done
!conda install basemap basemap-data-hires -y

Solving environment: done

# All requested packages already installed.



### Imports

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import torch
import os
import sys

# Fix error in case Basemap can't find epsg files, see https://github.com/matplotlib/basemap/issues/419
# You might need to adjust the path, e.g. your proj4-folder could have a slightly different name
os.environ['PROJ_LIB'] = "/home/andreas/anaconda3/pkgs/proj4-5.2.0-he6710b0_1/share/proj/"
sys.path.append("..")
%matplotlib inline

## Constants and Hyperparameters

In [3]:
#### Paths and configuration regarding encoder
BASE_PATH = "../../test/logs/Sun_Jun__9_16_15_40_201987405"
MODEL_BASE_PATH = os.path.join(BASE_PATH, "models")
CONFIG_PATH = os.path.join(BASE_PATH, "config.json")
EPOCH = 0 # TODO: Automatically use latest one
config = json.load(open(CONFIG_PATH, "rt"))


# Relevant information about data set
dataset = config['data']['name']
n_features = config['data'][dataset]['dims']
n_atoms = config['data'][dataset]['atoms']
n_edge_types = config['model']['n_edge_types']
timesteps = config['data']['timesteps']


# Set data location
DATA_PATH = os.path.abspath("") 

### Create Map to plot on
As our data set's weather stations are located in Spain, the below used coordinates represent a square fully comprising Spain.

In [4]:
from utils import id_2_loc, plot_interactions, edges_to_adj
from mpl_toolkits.basemap import Basemap

# Map to plot weather stations
map = Basemap(projection='lcc', resolution='l',
            width=1.5E6, height=1.5E6, 
            lat_0=40, lon_0=-4.5)

In [5]:
# Load data
data, configs, config_indices = np.load(DATA_PATH, allow_pickle=True)

IsADirectoryError: [Errno 21] Is a directory: '/home/andreas/Studium/Semester2/Lab/project-4/notebooks/weather'

# Extract Latent Graphs with Encoder

In [ ]:
# Load Encoder
from src.model.utils import load_weights, gen_fully_connected
from train import create_encoder
import torch

# Change to device('cuda:0') e.g. to run on GPU
device = torch.device('cpu')

# Load model as defined in config and 
encoder = create_encoder(config).to(device)
#load_weights(encoder, os.path.join(MODEL_BASE_PATH, f"encoder_epoch{EPOCH}.pt"))

# Move data to GPU if used
data_tensor = data.to(device)


rel_rec, rel_send = gen_fully_connected(n_atoms, device=device)

In [ ]:
from src.model.utils import gumbel_softmax

input = data_tensor[:, :, :timesteps, :]

logits = encoder(input, rel_rec, rel_send)
edges = gumbel_softmax(logits, tau=0.5, hard=True)
graphs = edges_to_adj(edges, n_atoms=n_atoms)

In [ ]:
# Plot interactions on Map
sample_id = 0

plot_interactions(configs[config_indices[sample_id]], graphs[0], map=map, station_locations=config_)

In [ ]:
# Visualize adjacency matrix
fig, axes = plt.subplots(nrows=1, ncols=n_edge_types)
for i in range(n_edge_types):
    im = axes[i].imshow(graphs[sample_id, i, :, :], cmap='gray', interpolation=None)
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.6)

## Let's see how the latent graph changes over time


In [ ]:
window_size = timesteps

dynamic_graphs = []

for step in range(0, 100 - timesteps): #data_tensor.size(-2) - timesteps):
    # Use sliding window to update latent graph over timesteps
    #input = data_tensor[:, :, step:step+timesteps, :]
    #logits = encoder(input, rel_rec, rel_send)
    #edges = gumbel_softmax(logits, tau=0.5, hard=True)
    edges = torch.rand((100, n_atoms * (n_atoms - 1), n_atoms))
    graphs = edges_to_adj(edges, n_atoms=n_atoms)
    dynamic_graphs.append(graphs)

In [ ]:
sample_id = 0

plot_interactions()